In [2]:
cd ImageRestoration/degradation_learning/learning_degradation/

/export/home2/capd0001/ImageRestoration/degradation_learning/learning_degradation


In [30]:
import torch
from torchvision import transforms
from PIL import Image

In [25]:
from basicsr.models.archs.encoder import ConvEncoder

In [26]:
net = ConvEncoder()

In [27]:
state_dict = torch.load("./checkpoints/net_E.pth")

In [28]:
net.load_state_dict(state_dict)

<All keys matched successfully>

In [33]:
img1 = Image.open('datasets/GOPRO/train/GOPR0372_07_00/blur/000047.png')
trans = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
img1 = trans(img1)

In [38]:
emb = net(img1.unsqueeze(0))

In [42]:
class ConditionalEmbedding(nn.Module):
    def __init__(self, num_labels:int, d_model:int, dim:int):
        assert d_model % 2 == 0
        super().__init__()
        self.condEmbedding = nn.Sequential(
            nn.Embedding(num_embeddings=num_labels + 1, embedding_dim=d_model, padding_idx=0),
            nn.Linear(d_model, dim),
            nn.SiLU(),
            nn.Linear(dim, dim),
        )

    def forward(self, t:torch.Tensor) -> torch.Tensor:
        emb = self.condEmbedding(t)
        return emb

In [ ]:
cemblayer = ConditionalEmbedding(10,32,params.modch).to(device)